## 前処理用のコード 第4版

号棟データのkobetsu1～4および現場データのhokakisei1～4について、trainとtest双方のデータに出現するものからダミー変数列を作成するようにする。
ただし、以下は保留。
- 現場データ：road2～4未反映

##  まだダミー変数列の統一が必要なことが判明した
- road1_hk
- toshikuiki
- kodochiku
- setsudo_hi
## ⇒ trainとtestを一つのデータにしてからダミー変数化、そのご分割することにする。

In [3]:
import numpy as np
import pandas as pd
import math

### 前処理するうえで考慮しなくてはいけない事項
- 号棟データの欠損を補完するために、現場データとジョインしてから号棟データの前処理をする。
- 号棟データのkobetsu1～4は、４列に出現するデータでダミー変数化する必要がある。
- 現場データのhokakisei1～4は、４列に出現するデータでダミー変数化する必要がある。

よって次の順番で処理する。
1. 現場データのtrainとtestを連結
1. 連結した現場データの欠損補完やhokakisei1～4のダミー変数化を行う
1. 号棟データのtrainを読みこみ、目的変数を分離しておく。
1. 号棟データのtestを読み込み、上記のtrain号棟データと連結
1. 連結済みの現場データと連結済み号棟データをジョイン
1. ジョイン済号棟データの欠損補完やkobetsu1～4のダミー変数化を行う
1. 号棟データのtrainとtestを分離
1. 号棟データのtrainから分離した真の目的変数から、学習で使用する単価面積を算出

### 追加のアイデア
1. 号棟データの接道方位(setsudo_hi) ... 東＋西のように２方向ある場合がある。東と西それぞれに分解すべき
1. 現場データのroad1～4の方角 ... ８方位のダミー変数にするのがよい

In [75]:
def create_list( df,cols ):
    names = set()
    for col in cols:
        for i in range(len(df)):
            name = df.at[i,col]
            if ( type(name) != float ):
                names.add(name)
    return sorted(list(names))

In [76]:
# 現場データのtrainとtestを連結
train = pd.read_csv("data/train_genba.tsv", sep='\t', index_col=0 )
test = pd.read_csv("data/test_genba.tsv", sep='\t', index_col=0)

In [77]:
# 連結した現場データの欠損補完やhokakisei1～4のダミー変数化を行う
genba = pd.concat([train,test],axis=0)
hokaise_list = create_list( genba, {'hokakisei1','hokakisei2','hokakisei3','hokakisei4'})
processed_genba = process_genba(genba, hokaisei_list)

NameError: name 'process_genba' is not defined

In [71]:
print(train.at[0,'jukyo'])
print(test.at[0,'jukyo'])
print(genba.at[0,'jukyo'])

埼玉県朝霞市泉水


KeyError: 0

In [72]:
type(train)
type(test)
type(genba)

pandas.core.frame.DataFrame

In [64]:
df = pd.concat([train,test],axis=0)

In [73]:
train.head()

,jukyo,chiseki_js_hb,chiseki_kb_hb,yoto1,yoto2,kempei1,kempei2,yoseki1,yoseki2,josui,...,bas_toho1,eki_kyori1,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2
pj_no,,,,,,,,,,,,,,,,,,,,,
0,埼玉県朝霞市泉水,109.26,109.10,工業地域,NaN,60,0.0,200,0.0,公営,...,徒歩,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,埼玉県川越市大字的場,136.15,136.00,第一種低層住居専用地域,NaN,60,0.0,200,0.0,公営,...,徒歩,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,埼玉県川口市安行原,86.92,86.52,第一種低層住居専用地域,NaN,50,0.0,100,0.0,私営,...,徒歩,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,埼玉県草加市松江,164.91,164.07,工業地域,NaN,60,0.0,200,0.0,公営,...,徒歩,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,埼玉県新座市野火止,403.58,403.58,第一種低層住居専用地域,NaN,50,0.0,80,0.0,公営,...,徒歩,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
test.head()

,pj_no,jukyo,chiseki_js_hb,chiseki_kb_hb,yoto1,yoto2,kempei1,kempei2,yoseki1,yoseki2,...,bas_toho1,eki_kyori1,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2
0,2781,埼玉県羽生市東,208.26,208.26,第一種低層住居専用地域,NaN,50,NaN,80,NaN,...,徒歩,21,NaN,NaN,NaN,NaN,徒歩,NaN,NaN,NaN
1,2782,埼玉県川越市大字寺尾,1369.35,1497.00,第一種低層住居専用地域,NaN,50,0.0,100,0.0,...,徒歩,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2783,埼玉県三郷市新和,110.28,110.28,準工業地域,NaN,60,NaN,200,NaN,...,徒歩,9,NaN,NaN,NaN,NaN,徒歩,NaN,NaN,NaN
3,2784,埼玉県志木市柏町,198.38,198.00,第一種中高層住居専用地域,NaN,60,0.0,200,0.0,...,徒歩,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2785,埼玉県さいたま市北区奈良町,229.45,229.45,第一種中高層住居専用地域,NaN,60,0.0,200,0.0,...,バス,13,奈良団地,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
genba.head()

,pj_no,jukyo,chiseki_js_hb,chiseki_kb_hb,yoto1,yoto2,kempei1,kempei2,yoseki1,yoseki2,...,bas_toho1,eki_kyori1,bastei_nm1,teiho1,rosen_nm2,eki_nm2,bas_toho2,eki_kyori2,bastei_nm2,teiho2
0,0,埼玉県朝霞市泉水,109.26,109.10,工業地域,NaN,60,0.0,200,0.0,...,徒歩,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,埼玉県川越市大字的場,136.15,136.00,第一種低層住居専用地域,NaN,60,0.0,200,0.0,...,徒歩,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,埼玉県川口市安行原,86.92,86.52,第一種低層住居専用地域,NaN,50,0.0,100,0.0,...,徒歩,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,埼玉県草加市松江,164.91,164.07,工業地域,NaN,60,0.0,200,0.0,...,徒歩,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,埼玉県新座市野火止,403.58,403.58,第一種低層住居専用地域,NaN,50,0.0,80,0.0,...,徒歩,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
train = pd.read_csv("data/train_genba.tsv", sep='\t')
test = pd.read_csv("data/test_genba.tsv",sep='\t')
hokakisei_list = create_list( train, test, {'hokakisei1','hokakisei2','hokakisei3','hokakisei4'})
train = pd.read_csv("data/train_goto.tsv",sep='\t')
test = pd.read_csv("data/train_goto.tsv",sep='\t')
kobetsu_list = create_list( train, test, ['kobetsu1','kobetsu2','kobetsu3','kobetsu4'])

In [20]:
train_genba = pd.read_csv("data/train_genba.tsv",sep='\t')
genba = preprocess_genba(train_genba, hokakisei_list)
genba.to_csv('data/processed_train_genba.csv', index=False)

train_goto = pd.read_csv("data/train_goto.tsv",sep='\t')
train_goto = train_goto.merge(genba, how='left', on='pj_no')
goto = preprocess_goto(train_goto, kobetsu_list)

In [21]:
#
# 契約金額（keiyaku_pr）が目的変数だが、公示価格等と相関があるとおもわれるので
# 土地面積(tc_mseki)で割ったものを目的変数とする。
#
goto['tc_tanka']=goto['keiyaku_pr']/goto['tc_mseki']

#
# あとで推定契約金額と比較するために、契約金額を別に描き出しておく
#
keiyaku = pd.DataFrame(goto[['id','keiyaku_pr']])
keiyaku.to_csv('data/processed_keiyaku_train_goto.csv', index=False)

# 元々の目的変数も削除しておく
goto = goto.drop('keiyaku_pr',axis=1)
goto.to_csv('data/processed_train_goto.csv',index=False)

In [22]:
#
# テスト用データの前処理
#
test_genba = pd.read_csv("data/test_genba.tsv",sep='\t')
genba = preprocess_genba(test_genba,hokakisei_list)
genba.to_csv("data/processed_test_genba.csv",index=False)

test_goto = pd.read_csv("data/test_goto.tsv",sep='\t')
test_goto = test_goto.merge(genba, how='left', on='pj_no')
goto = preprocess_goto(test_goto,kobetsu_list)
goto.to_csv("data/processed_test_goto.csv",index=False)

In [11]:
def preprocess_goto(goto, kobetsu_cols):
    # コピーしてから加工する
    df = goto.copy()
    
    # 削除対象のカラム名を追加していくリスト
    del_list = []
    
    # 階数・プラン(levelplan)
    # 求めるものは土地の販売価格なので不要とする。
    del_list.append("levelplan")
    
    # 幅員(fukuin)
    # 欠損および０の場合には、genbaデータと結合したroad1_fiの値をコピーする
    df['fukuin'] = df['fukuin'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'fukuin'] ==0 ) :
            df.at[i,'fukuin'] = df.at[i,'road1_fi']
    
    # 道路状況(road_st)
    # 以下のように値にマップする。欠損は"問題なし”にマップすることにする。
    df['road_st'] = df['road_st'].fillna("問題なし")
    df['road_st'] = df['road_st'].map({'未舗装':0, '問題なし':1, '歩道あり':1, '歩道+緑地帯あり':2})
    
    # 間口(magutchi)
    # 幅員と同じように補完する。
    df['magutchi'] = df['magutchi'].fillna(0)
    for i in range(len(df)):
        if ( df.at[i,'magutchi'] ==0 ) :
            df.at[i,'magutchi'] = df.at[i,'road1_mg']    
    
    # 接道方位	(setsudo_hi)
    # これも幅員と同じように補完してから、ダミー変数化する。
    df['setsudo_hi' ] = df['setsudo_hi'].fillna('unknown')
    for i in range(len(df)):
        if ( df.at[i,'setsudo_hi']=='unknown'):
            df.at[i,'setsudo_hi']=df.at[i,'road1_hk']
    
    df = pd.merge(df, pd.get_dummies(df[['setsudo_hi']],drop_first = False), left_index = True, right_index = True)
    del_list.append('setsudo_hi')
    del_list.append('road1_hk')
    
    # 接道形状(setsudo_kj)
    # trainデータでは、'良い'が一番多いので、それで補完し、ダミー変数化する。
    df['setsudo_kj' ] = df['setsudo_kj'].fillna('良い')
    df = pd.merge(df, pd.get_dummies(df[['setsudo_kj']],drop_first = False), left_index = True, right_index = True)
    del_list.append('setsudo_kj')
    
    # 地型(jigata)
    # trainデータで一番多い'整形地’で補完し、ダミー変数化する。
    df['jigata'] = df['jigata'].fillna('整形地')
    df = pd.merge(df, pd.get_dummies(df[['jigata']],drop_first = False), left_index = True, right_index = True)
    del_list.append('jigata')
    
    # 日当たり(hiatari)
    # trainデータで多い'普通'で補完し、ダミー変数化する
    # genbaデータから補完できるかもしれない。
    df['hiatari'] = df['hiatari'].fillna('普通')
    df = pd.merge(df, pd.get_dummies(df[['hiatari']],drop_first = False), left_index = True, right_index = True)
    del_list.append('hiatari')
    
    # 個別要因①〜④
    # ダミー変数化する  
    kobetsu_map = {'信号近い':'信号前'}
    df['kobetsu1']=df['kobetsu1'].replace(kobetsu_map)
    df['kobetsu2']=df['kobetsu2'].replace(kobetsu_map)
    df['kobetsu3']=df['kobetsu3'].replace(kobetsu_map)
    df['kobetsu4']=df['kobetsu4'].replace(kobetsu_map)

    # ステップ２：
    kobetsu_dict = {}
    for x in kobetsu_cols:
        kobetsu_dict[x] = np.zeros(len(df), dtype=np.int)

    # ステップ３：
    for i in range(len(df)):
        name = df.at[i,'kobetsu1']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1
        name = df.at[i,'kobetsu2']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1
        name = df.at[i,'kobetsu3']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1
        name = df.at[i,'kobetsu4']
        if ( type(name) != float):
            kobetsu= kobetsu_dict[name]
            kobetsu[i]=1

    # 元のDataFrameとダミー変数化したものを連結
    df = pd.concat([df, pd.DataFrame(kobetsu_dict)], axis=1)
    
    # 不要になった列を削除する
    del_list.append('kobetsu1')
    del_list.append('kobetsu2')
    del_list.append('kobetsu3')
    del_list.append('kobetsu4')
    
    df = df.drop(del_list, axis=1)
    
    return df

In [18]:
def preprocess_genba(genba, hokakisei_cols):
    df = genba.copy()
    
    # jukyoは使用しないのでを削除
    df = df.drop('jukyo',axis=1)
    
    # 地積公募値には欠損あり(chiseki_kb_hb)
    # 実測値で補完する
    df['chiseki_kb_hb'] = df['chiseki_kb_hb'].fillna(-99)
    for i in range(len(df)):
        if ( df.at[i,'chiseki_kb_hb'] == -99 ):      
            df.at[i,'chiseki_kb_hb'] = df.at[i,'chiseki_js_hb']
    
    # 地積の公募値と実測値の差を特徴量に追加
    df['chiseki_diff'] = df['chiseki_kb_hb']-df['chiseki_js_hb']
    
    # 二つの用途から、一つの用途、建ぺい率、容積率を返却する
    # yotoはダミー変数化する
    df[['yoto','kempei','yoseki']] = df.apply(select_yoto, axis=1)
    df = pd.merge(df, pd.get_dummies(df[['yoto']],drop_first = False), left_index = True, right_index = True)
    
    # 上水種別を数値化
    df['josui'] = df.apply(lambda x: 1 if x['josui']=='公営' else 0, axis=1)
    
    # 下水種別を数値化
    df['gesui'] = df.apply(lambda x: 1 if x['gesui']=='公共下水' else 0, axis=1)
    
    # ガス種別を数値化
    df['gas'] = df.apply(lambda x: 2 if x['gas'] =='都市ガス' else 1 if x['gas'] == '集中プロパン' else 0, axis=1)
    
    # 雨水（usui）をダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['usui']],drop_first = False), left_index = True, right_index = True)
    
    # 欠損値処理
    # 建て売り数 0で補完（trainデータのID:726は、一件だけの販売なので、他の建て売り数は０で正しい
    df = df.fillna({'tateuri_su': 0, 'tochiuri_su':0 , 'joken_su':0 })
    
    # 同現場での階数ごとの販売数は、gotoデータとつきあわせると正確に出そうだが、とりあえず０で補完する。
    df = df.fillna({'hy1f_date_su':0, 'hy2f_date_su':0, 'hy3f_date_su':0})

    # 家屋面積(平米) (kaoku_hb)
    # 欠損しているところは、家屋有無が(無)なので、0で補完する出正しい。
    # 道路面積(mseki_rd_hb), ゴミ、公園等面積(mseki_dp_hb)は大半がゼロなのでゼロで補完しておく
    df['kaoku_hb'] = df['kaoku_hb'].fillna(0)
    df = df.fillna({'kaoku_hb':0, 'mseki_rd_hb':0, 'mseki_dp_hb':0})
    
    # 道路
    # 扱いが難しい。road1〜road4まであるが、road1だけのものもあれば、road1〜4まで４つあるものまで。
    # とりあえず、road1だけを採用し、接する道路数を表す項目を追加することにする。
    # Future TODO
    # road2〜road4のデータの採用のしかた
    
    # road1
    # road1_hkとroad1_sbをダミー変数化する。
    # road2〜road4の有無で、道路数を表すroad_numを作る
    # TODO: road2〜road4を削除
    df = pd.merge(df, pd.get_dummies(df[['road1_hk','road1_sb']],drop_first = False), left_index = True, right_index = True)
    df['road_num']=df.apply(lambda x: 4 if x['road4_fi']!=0 else 3 if x['road3_fi']!=0 else 2 if x['road2_fi']!=0 else 1, axis=1)
    
    #kaoku_umとyheki_umuを0と1で置き換え
    df['kaoku_um']=df.apply(lambda x: 0 if x['kaoku_um']=="（無）" else 1, axis=1)
    df['yheki_umu']=df.apply(lambda x: 0 if x['yheki_umu']=="（無）" else 1, axis=1)
    
    #yheki_yohiを0と1で置き換え
    df['yheki_yohi']=df.apply(lambda x: 0 if x['yheki_yohi']=="（不要）" else 1, axis=1)
    
    #kborjsを0と1で置き換え。"公募"に変換ミスがあって、３通りあるため、「実測」で値を決める
    df['kborjs']=df.apply(lambda x: 0 if x['kborjs']=='実測' else 1, axis=1)
    
    # 引渡の状態 hw_status
    # 善し悪しを表しそうなので数値化したいが、とりあえずはダミー変数化する。
    df = pd.merge(df, pd.get_dummies(df[['hw_status']],drop_first = False), left_index = True, right_index = True)
    
    #都市計画区域別 toshikuiki1とtoshikuiki２　toshikuiki2があればそちらを採用する。
    #一度toshikuiki2がカラの場合にtoshikuiki(新設)に特殊な値を入れて比較しないとうまくいかない。他に方法がありそうだが
    df['toshikuiki']=df['toshikuiki2'].fillna('empty')
    df['toshikuiki'] = df.apply(lambda x: x['toshikuiki1'] if x['toshikuiki']=="empty" else x['toshikuiki2'], axis=1)
    df = pd.merge(df, pd.get_dummies(df[['toshikuiki']],drop_first = False), left_index = True, right_index = True)
    
    # 高度地区(kodochiku) ダミー変数化する
    df = pd.merge(df, pd.get_dummies(df[['kodochiku']],drop_first = False), left_index = True, right_index = True)
    
    # chikukeikaku, keikakuroad, kaihatsukyoka, t53kyoka, hokakyokaを0,1に変換
    df['chikukeikaku']=df.apply(lambda x: 0 if x['chikukeikaku']=="（無）" else 1, axis=1)
    df['keikakuroad']=df.apply(lambda x: 0 if x['keikakuroad']=="（無）" else 1, axis=1)
    df['kaihatsukyoka']=df.apply(lambda x: 0 if x['kaihatsukyoka']=="（不要）" else 1, axis=1)
    df['t53kyoka']=df.apply(lambda x: 0 if x['t53kyoka']=="（不要）" else 1, axis=1)
    df['hokakyoka']=df.apply(lambda x: 0 if x['hokakyoka']=="（不要）" else 1, axis=1)
    
    # 防火地区をダミー変数化
    df = pd.merge(df, pd.get_dummies(df[['bokachiiki']],drop_first = False), left_index = True, right_index = True)
    
    # 最低敷地面積(minmenseki) 欠損がある。ゼロで補完する
    df['minmenseki'] = df['minmenseki'].fillna(0)
    
    # その他規制①～④(hokakisei1～4)
    # 四つの列でダミー変数化する必要あり。
    
    # まず、表記を一意にしておき...
    kisei_map = {'公有地拡大推進法':'公拡法', '文化財保護法':'埋蔵文化財', '文化財保護法（埋蔵文化財）':'埋蔵文化財', '景観地区':'景観法','東日本大震災復興特別区域法':'東日本震災復興特', '農地法届出要':'農地法', '43条第1項ただし書き許可':'43条許可'}
    df['hokakisei1']=df['hokakisei1'].replace(kisei_map)
    df['hokakisei2']=df['hokakisei2'].replace(kisei_map)
    df['hokakisei3']=df['hokakisei3'].replace(kisei_map)
    df['hokakisei4']=df['hokakisei4'].replace(kisei_map)
       
    # カラのダミー変数列をつくっておいて...
    kisei_dict = {}
    for x in hokakisei_cols:
        kisei_dict[x] = np.zeros(len(df), dtype=np.int)
    
    # ダミー変数列を作って...
    for i in range(len(df)):
        name = df.at[i,'hokakisei1']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
        name = df.at[i,'hokakisei2']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
        name = df.at[i,'hokakisei3']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
        name = df.at[i,'hokakisei4']
        if ( type(name) != float):
            kisei = kisei_dict[name]
            kisei[i]=1
    
    # 元のDataFrameとダミー変数化したものを連結して..
    df = pd.concat([df, pd.DataFrame(kisei_dict)], axis=1)
    
    # 不要になったhokakisei1～hokakisei4を削除する
    df = df.drop(['hokakisei1','hokakisei2','hokakisei3','hokakisei4'], axis=1)
    
    # kinshijikoを0,1に変換
    df['kinshijiko']=df.apply(lambda x: 0 if x['kinshijiko']=="（無）" else 1, axis=1)
    
    # 路線価格(rosenka_hb)
    # 公示価格(koji_hb)
    # 基準地価(kijun_hb)
    # 路線価格と基準地価には欠損(値が0)あり。公示価格は欠損なし。
    # 公示価格だけを残す。
    df = df.drop(['rosenka_hb','kijun_hb'],axis=1)

    # 有効宅地面積(mseki_yt_hb)
    # 道路(mseki_rd_hb)
    # ゴミ置場・公園等(mseki_dp_hb)
    # 相関不明。そのまま残しておく。
    
    # 土地最小面積～建物平均面積
    # tc_mseki_min_hb～tt_mseki_avg_hb
    # 地価に関係ないと思われるので落とす。
    df = df.drop(['tc_mseki_min_hb','tc_mseki_max_hb','tt_mseki_min_hb','tt_mseki_max_hb','tc_mseki_avg_hb','tt_mseki_avg_hb'], axis=1)
    
    # 幅員4m未満道路の通過(fi4m_yohi)と幅員3m以上4m未満道路の通過距離(fi4m_kyori)
    # fi4m_yoshiが否なら、fi4m_kyoriが０。fi3m...の項目も同様。
    # よって、fi4m_yohiとfi3m_yohiは削除する。
    # かつ、fi4m_kyori, fi3m_kyoriが欠損しているところは、yohiがちゃんと否になっているので、0で埋める。
    df = df.drop(['fi4m_yohi','fi3m_yohi'],axis=1)
    df['fi4m_kyori']=df['fi4m_kyori'].fillna(0)
    df['fi3m_kyori']=df['fi3m_kyori'].fillna(0)
    
    # バス要否、バス本数
    # 後続の交通手段のところで、バスを使うことになっていないにもかかわらず、要否が要になっているデータがあり
    # 信ぴょう性に欠けるので、削除する。
    df = df.drop(['bus_yohi','bus_hon'], axis=1)
    
    # コンビニ有無(sho_conv)～騒音有無(shu_soon)
    # ２値データ。ありは「〇」。これを0/1に変換する。
    df['sho_conv'] = df['sho_conv'].map({'○':1, np.NaN:0})
    df['sho_super'] = df['sho_super'].map({'○':1, np.NaN:0})
    df['sho_shoten'] = df['sho_shoten'].map({'○':1, np.NaN:0})    
    df['sho_market'] = df['sho_market'].map({'○':1, np.NaN:0})
    df['shu_jutaku'] = df['shu_jutaku'].map({'○':1, np.NaN:0})
    df['shu_park'] = df['shu_park'].map({'○':1, np.NaN:0})
    df['shu_shop'] = df['shu_shop'].map({'○':1, np.NaN:0})
    df['shu_factory'] = df['shu_factory'].map({'○':1, np.NaN:0})
    df['shu_hvline'] = df['shu_hvline'].map({'○':1, np.NaN:0})
    df['shu_tower'] = df['shu_tower'].map({'○':1, np.NaN:0})
    df['shu_bochi'] = df['shu_bochi'].map({'○':1, np.NaN:0})
    df['shu_sogi'] = df['shu_sogi'].map({'○':1, np.NaN:0})
    df['shu_zoki'] = df['shu_zoki'].map({'○':1, np.NaN:0})
    df['shu_kokyo'] = df['shu_kokyo'].map({'○':1, np.NaN:0})
    df['shu_highway'] = df['shu_highway'].map({'○':1, np.NaN:0})
    df['shu_kaido'] = df['shu_kaido'].map({'○':1, np.NaN:0})
    df['shu_line_ari'] = df['shu_line_ari'].map({'○':1, np.NaN:0})
    df['shu_line_nashi'] = df['shu_line_nashi'].map({'○':1, np.NaN:0})
    df['shu_soon'] = df['shu_soon'].map({'○':1, np.NaN:0})
    
    # 幼稚園徒歩/小学校徒歩/中学校徒歩(gk_yoc_tm/gk_sho_tm/gk_chu_tm)
    # 欠損もなく異常値もないため、そのままにする。
    
    # 隣接物件東戸建（2F以下）～隣接物件北田畑
    # ２値データ。ありは「〇」。これを0/1に変換する
    df['rs_e_kdate2'] = df['rs_e_kdate2'].map({'○':1, np.NaN:0})
    df['rs_e_kdate3'] = df['rs_e_kdate3'].map({'○':1, np.NaN:0})
    df['rs_e_parking'] = df['rs_e_parking'].map({'○':1, np.NaN:0})
    df['rs_e_zoki'] = df['rs_e_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_e_m_ari'] = df['rs_e_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_e_m_nashi'] = df['rs_e_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_e_tahata'] = df['rs_e_tahata'].map({'○':1, np.NaN:0}) 

    df['rs_w_kdate2'] = df['rs_w_kdate2'].map({'○':1, np.NaN:0})
    df['rs_w_kdate3'] = df['rs_w_kdate3'].map({'○':1, np.NaN:0})
    df['rs_w_parking'] = df['rs_w_parking'].map({'○':1, np.NaN:0})
    df['rs_w_zoki'] = df['rs_w_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_w_m_ari'] = df['rs_w_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_w_m_nashi'] = df['rs_w_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_w_tahata'] = df['rs_w_tahata'].map({'○':1, np.NaN:0}) 
    
    df['rs_s_kdate2'] = df['rs_s_kdate2'].map({'○':1, np.NaN:0})
    df['rs_s_kdate3'] = df['rs_s_kdate3'].map({'○':1, np.NaN:0})
    df['rs_s_parking'] = df['rs_s_parking'].map({'○':1, np.NaN:0})
    df['rs_s_zoki'] = df['rs_s_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_s_m_ari'] = df['rs_s_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_s_m_nashi'] = df['rs_s_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_s_tahata'] = df['rs_s_tahata'].map({'○':1, np.NaN:0}) 
    
    df['rs_n_kdate2'] = df['rs_n_kdate2'].map({'○':1, np.NaN:0})
    df['rs_n_kdate3'] = df['rs_n_kdate3'].map({'○':1, np.NaN:0})
    df['rs_n_parking'] = df['rs_n_parking'].map({'○':1, np.NaN:0})
    df['rs_n_zoki'] = df['rs_n_zoki'].map({'○':1, np.NaN:0}) 
    df['rs_n_m_ari'] = df['rs_n_m_ari'].map({'○':1, np.NaN:0}) 
    df['rs_n_m_nashi'] = df['rs_n_m_nashi'].map({'○':1, np.NaN:0}) 
    df['rs_n_tahata'] = df['rs_n_tahata'].map({'○':1, np.NaN:0}) 

    # 路線(rosen_nm1/rosen_nm2)、駅(eki_nm1/eki_nm2)
    # 残してある公示価格に反映されていると考えて、ここでは削除する
    df[['bus', 'walk']] = df.apply(calculateLogistics, axis=1)
    df=df.drop(['rosen_nm1','eki_nm1','bas_toho1','eki_kyori1','bastei_nm1','teiho1','rosen_nm2','eki_nm2','bas_toho2','eki_kyori2','bastei_nm2','teiho2'],axis=1)

    # 最後に不要になった列を削除する
    # road_hkは、gotoデータと結合した後に補完に使うので、個々では削除しないでおく。
    df=df.drop(['yoto1','kempei1','yoseki1','yoto2','kempei2','yoseki2','road1_sb','toshikuiki1','yoto'],axis=1)
    df=df.drop(['usui','road2_hk','road2_sb','road2_fi','road2_mg','road3_hk','road3_sb','road3_fi','road3_mg','road4_hk','road4_sb','road4_fi','road4_mg'],axis=1)
    df=df.drop(['hw_status','toshikuiki','toshikuiki2','toshikuiki','kodochiku','bokachiiki'],axis=1)
    
    # testデータの項目gc_yoc_tmに欠損あり。
    # 平均値で欠損補完する
    df['gk_yoc_tm'] = df['gk_yoc_tm'].fillna(df['gk_yoc_tm'].mean())
        
    return df

def select_yoto(x):
    # yoto2はNaNを含む
    # 値がない場合、typeはfloatで、データがある場合にはstrになる様子。
    # そこで、型がstrかどうかで、値の有無を判定する。
    if ( type(x['yoto2']) == str ):
        yoto =x['yoto2']
        kempei = x['kempei2']
        yoseki =x['yoseki2']
    else:
        yoto =x['yoto1']
        kempei =x['kempei1']
        yoseki =x['yoseki1'] 
    return pd.Series([yoto, kempei, yoseki])

# 駅から徒歩、バス等の交通手段を、バスと徒歩の二つの利用時間に変換する
# バスを使わない場合には、busを0にすることで対応。

def calculateLogistics(x):
    if ( x['bas_toho1'] == 'バス' ):
        bus = x['eki_kyori1']
        walk = x['teiho1']
    else:
        bus = 0
        walk = x['eki_kyori1']
    
    return pd.Series([bus, walk])

# 法規制の処理用関数
def put_name(col, name):
    if ( type(name) != float ):
        col.add(name)

In [19]:
preprocess_genba

<function __main__.preprocess_genba>

### 様々な関数

#### data_definition.txtによると、用途２がある場合には用途２を使う、とあった。
#### 当初は、建ぺい率の大きい方のyotoを採用し、yoto, kempei, yosekiからなるSeriesを返却する　とした。